# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_for_csv = "../WeatherPy/output_data/city_weather.csv"
weather_df_uncleaned = pd.read_csv(weather_for_csv)

# weather_df_uncleaned.dtypes (change humidity to float)
weather_df_uncleaned

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Esperance,-33.87,121.90,55.40,87,90,3.36,AU,1609705288
1,Nikolskoye,59.70,30.79,28.40,86,75,2.24,RU,1609705505
2,Ushuaia,-54.80,-68.30,51.80,50,75,12.75,AR,1609705328
3,Evensk,61.95,159.23,1.31,87,100,13.22,RU,1609705506
4,Paamiut,61.99,-49.67,26.35,91,100,13.24,GL,1609705410
...,...,...,...,...,...,...,...,...,...
654,Sukhobezvodnoye,57.05,44.89,23.32,98,100,5.99,RU,1609705657
655,Thakurdwara,29.20,78.85,59.83,65,4,6.46,IN,1609705657
656,Victoria Point,-27.58,153.30,78.01,96,100,5.12,AU,1609705657
657,Plettenberg Bay,-34.05,23.37,70.27,92,60,4.54,ZA,1609705657


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
gmaps.configure(api_key=g_key)

# set up lat/lng as locs and hum as weight
locations = weather_df_uncleaned[["Lat", "Lng"]]
humidity = weather_df_uncleaned["Humidity"]

In [25]:
# set up heatmap then heatlayer to add over map
heatmap = gmaps.figure(center=(20, 10), zoom_level=2)

heatlayer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=2.5)

heatmap.add_layer(heatlayer)
heatmap

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [44]:
best_weather_df = weather_df_uncleaned

best_weather_df = best_weather_df.loc[(best_weather_df["Max Temp"] <= 85) & (best_weather_df["Max Temp"] > 72)]
best_weather_df = best_weather_df.loc[best_weather_df["Wind Speed"] <= 5]
best_weather_df = best_weather_df.loc[best_weather_df["Cloudiness"] <= 30]

best_weather_df = best_weather_df.dropna()
best_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
49,Hilo,19.73,-155.09,75.99,73,20,3.36,US,1609705248
84,Gao,16.64,1.64,76.64,20,0,4.74,ML,1609705225
137,Sangmélima,2.93,11.98,72.45,72,1,2.44,CM,1609705535
200,Padang,-0.95,100.35,73.40,94,20,1.12,ID,1609705475
223,Bosaso,11.28,49.18,74.17,73,7,3.58,SO,1609705555
240,Vélingara,13.15,-14.12,84.47,18,0,4.16,SN,1609705559
259,Honiara,-9.43,159.95,73.40,100,20,2.24,SB,1609705523
342,Noumea,-22.28,166.46,81.00,94,0,2.24,NC,1609705582
388,Manjacaze,-24.71,33.88,73.89,93,0,4.99,MZ,1609705592
404,Berbera,10.44,45.01,75.11,71,0,4.12,SO,1609705596


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [48]:
hotel_df = best_weather_df

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# set params before for loop
params = {"radius": 5000, 
          "type": "lodging", 
          "key": g_key,}

# loop thru last df, get lat and lng, create json response, 
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    # params['location'] will allow loop to find specific lats/lngs for each loop iteration (nearest restr. activity)
    params['location'] = f"{lat}, {lng}"
    
    # set json resp.
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    # now place results into a new column "Hotel Name"
    
    try:
        print(f"Closest hotel is {results[0]['name']} for City {index}: {row['City']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        print("----------")

    except:
        print("Sorry, result can not be found... Moving to next City.")

Closest hotel is Hilo Hawaiian Hotel for City 49: Hilo.
----------
Sorry, result can not be found... Moving to next City.
Closest hotel is gamestar hotel for City 137: Sangmélima.
----------
Closest hotel is Mercure Padang for City 200: Padang.
----------
Closest hotel is Red Sea Hotel for City 223: Bosaso.
----------
Closest hotel is Hotel Najla for City 240: Vélingara.
----------
Closest hotel is Heritage Park Hotel for City 259: Honiara.
----------
Closest hotel is Hôtel Le Lagon for City 342: Noumea.
----------
Closest hotel is Hotel Gani for City 388: Manjacaze.
----------
Closest hotel is Damal Hotel for City 404: Berbera.
----------
Closest hotel is Komenan brou roseline for City 434: Daoukro.
----------
Closest hotel is Hotel Residency Tower for City 517: Trivandrum.
----------
Closest hotel is Hôtel des Chasses for City 540: Sarh.
----------
Closest hotel is Benin Metropole Hotel for City 601: Parakou.
----------
Closest hotel is Villa Orphee for City 634: Les Cayes.
---------

In [49]:
hotel_df = hotel_df.dropna()
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
49,Hilo,19.73,-155.09,75.99,73,20,3.36,US,1609705248,Hilo Hawaiian Hotel
137,Sangmélima,2.93,11.98,72.45,72,1,2.44,CM,1609705535,gamestar hotel
200,Padang,-0.95,100.35,73.40,94,20,1.12,ID,1609705475,Mercure Padang
223,Bosaso,11.28,49.18,74.17,73,7,3.58,SO,1609705555,Red Sea Hotel
240,Vélingara,13.15,-14.12,84.47,18,0,4.16,SN,1609705559,Hotel Najla
259,Honiara,-9.43,159.95,73.40,100,20,2.24,SB,1609705523,Heritage Park Hotel
342,Noumea,-22.28,166.46,81.00,94,0,2.24,NC,1609705582,Hôtel Le Lagon
388,Manjacaze,-24.71,33.88,73.89,93,0,4.99,MZ,1609705592,Hotel Gani
404,Berbera,10.44,45.01,75.11,71,0,4.12,SO,1609705596,Damal Hotel
434,Daoukro,7.06,-3.96,78.98,72,6,3.56,CI,1609705603,Komenan brou roseline


In [50]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [51]:
# Add marker layer ontop of heat map
# markerlayer = gmaps.marker_layer(locations)
marker = gmaps.marker_layer(locations, info_box_content=hotel_info,)
heatmap.add_layer(marker)

# Display figure
heatmap

Figure(layout=FigureLayout(height='420px'))